In [2]:
#Import the libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [3]:
#Loading mnist data and splitting the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step 
(60000, 28, 28) (60000,)


In [4]:
#Data Pre-processing
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [6]:
#mnist has 10 classes from 0 to 9 
num_classes = 10

#Converting class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
#Implementing the CNN model
#Create the model
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

C:\Users\VARUN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
#Train the model
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 63s 128ms/step - accuracy: 0.1560 - loss: 2.2828 - val_accuracy: 0.3201 - val_loss: 2.2504
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 61s 129ms/step - accuracy: 0.2858 - loss: 2.2299 - val_accuracy: 0.4790 - val_loss: 2.1859
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 61s 130ms/step - accuracy: 0.3982 - loss: 2.1621 - val_accuracy: 0.5758 - val_loss: 2.0997
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 130ms/step - accuracy: 0.4764 - loss: 2.0677 - val_accuracy: 0.6363 - val_loss: 1.9773
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 60s 128ms/step - accuracy: 0.5388 - loss: 1.9316 - val_accuracy: 0.6789 - val_loss: 1.8024
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 60s 129ms/step - accuracy: 0.5897 - loss: 1.7501 - val_accuracy: 0.7346 - val_loss: 1.5735
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 60s 128ms/step - accuracy: 0.6359 - loss: 1.5340 - val_accuracy: 0.7778 - val_loss: 1.3197
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 60s 129ms/step - accuracy: 0.6713 - loss: 1

In [9]:
#Saving the model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [10]:
#Evaluating the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7804424166679382
Test accuracy: 0.8327000141143799


In [11]:
#GUI to predict digits
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

In [12]:
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [26]:
import tkinter as tk
from tkinter import W
import win32gui
from PIL import ImageGrab
from IPython import display
import time

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.withdraw()  # Hide the window initially
        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self._safe_classify) 
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)
        
    def _safe_classify(self):
        """Wrapper to catch and display exceptions"""
        try:
            self.classify_handwriting()
        except Exception as e:
            print(f"Error in classify_handwriting: {str(e)}")
            self.label.configure(text=f"Error: {str(e)[:10]}...")
    
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        im = ImageGrab.grab(rect)
        
        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit)+', '+str(int(acc*100))+'%')
    
    def clear_all(self):
        self.canvas.delete("all")
        self.label.configure(text="Thinking..")
        
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x-r, self.y-r, self.x+r, self.y+r, fill='black')
        
    def show(self):
        """Show the window and run a brief mainloop"""
        self.deiconify()  # Show the window
        self.update()

# Create a new cell with just this code:
app = App()
app.show()
print("App is now running. Close the window when finished.")

App is now running. Close the window when finished.
